In [ ]:
import numpy as np
from tensorflow import keras

fashion_mnist = keras.datasets.fashion_mnist
(train_full_X, train_full_y), (test_X, test_y) = fashion_mnist.load_data()

In [ ]:
train_full_X = train_full_X.reshape(train_full_X.shape[0], 28, 28, 1)
test_X = test_X.reshape(test_X.shape[0], 28, 28, 1)

# Convert the images into 3 channels
train_full_X = np.repeat(train_full_X, 3, axis=3)
test_X = np.repeat(test_X, 3, axis=3)

In [ ]:
print("Train full X shape:", train_full_X.shape)
print("Test X shape:", test_X.shape)

Train full X shape: (60000, 28, 28, 3)
Test X shape: (10000, 28, 28, 3)


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Resizing

# Визначення моделі VGG16 з виключенням верхніх шарів
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Заморожування шарів VGG16 для запобігання їх оновленню під час тренування
vgg.trainable = False

# Створення нової моделі з використанням VGG16 як бази
base_model = Sequential([
    InputLayer(input_shape=(28, 28, 3)),
    Resizing(224, 224, interpolation='bilinear'),  # Зміна розміру зображень до 224x224
    vgg
])

# Виведення підсумку моделі
base_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
Total params: 14714688 (56.13 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

# Попередня обробка даних перед екстракцією ознак
train_full_X_processed = preprocess_input(train_full_X)
test_X_processed = preprocess_input(test_X)

# Екстракція ознак
train_features = base_model.predict(train_full_X_processed, batch_size=32, verbose=1)
test_features = base_model.predict(test_X_processed, batch_size=32, verbose=1)

# Перевірка розміру екстрактованих ознак
print("Розмір екстрактованих ознак для тренувального набору:", train_features.shape)
print("Розмір екстрактованих ознак для тестового набору:", test_features.shape)

In [ ]:
import numpy as np

# Збереження ознак тренувального набору
#np.save('train_features.npy', train_features)

# Збереження ознак тестового набору
#np.save('test_features.npy', test_features)


# Завантаження ознак тренувального набору
train_features = np.load('train_features.npy')

# Завантаження ознак тестового набору
test_features = np.load('test_features.npy')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Створення моделі для класифікації
model = Sequential([
    Flatten(input_shape=(7, 7, 512)),  # Вирівнювання ознак
    Dense(256, activation='relu'),     # Повністю з'єднаний шар
    Dropout(0.5),                      # Dropout для зменшення перенавчання
    Dense(10, activation='softmax')    # Вихідний шар для 10 класів
])

# Компіляція моделі
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Виведення підсумку моделі
model.summary()

# Тренування моделі
history = model.fit(train_features, train_full_y,
                    validation_split=0.2,
                    epochs=20,
                    batch_size=256,
                    verbose=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 6425354 (24.51 MB)
Trainable params: 6425354 (24.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20


188/188 [==============================] - 14s 69ms/step - loss: 0.8894 - accuracy: 0.8111 - val_loss: 0.3028 - val_accuracy: 0.8967


In [ ]:
# Оцінка моделі на тестовому наборі
test_loss, test_accuracy = model.evaluate(test_features, test_y, verbose=2)
print('\nТочність на тестовому наборі:', test_accuracy)

313/313 - 1s - loss: 0.3198 - accuracy: 0.9271 - 1s/epoch - 3ms/step

Точність на тестовому наборі: 0.9271000027656555


## Висновки по частині 2

Згорткова нейромережа, що використовує VGG16 в якості згорткової основи показала кращі результати порівняно з багатошаровою мережею з першого завдання. Точність скалала 92,7% проти 91,49%.

Параметри, використані для отримання результатів:

1. Архітектура моделі: Використання згорткової нейромережі VGG16 як базової моделі з виключенням верхніх шарів.
2. Замороження ваг: Шари VGG16 були заморожені, щоб уникнути їх оновлення під час тренування.
3. Попередня обробка даних: вхідні дані були оброблені функцією preprocess_input з пакету tensorflow.keras.applications.vgg16.
4. Розмір зображень: зображення були змінені до розміру 224x224 за допомогою шару Resizing (хоча було достатньо 32*32, швидкість роботи моделі значно знизилась, але результат був отриманий кращий).
5. Параметри компіляції моделі: були використані оптимізатор Adam з коефіцієнтом навчання 0.0001, функція втрат 'sparse_categorical_crossentropy', метрика точності.
6. Параметри тренування: тренування проводилося протягом 20 епох з розміром пакету 256 та розділеним навчальним набором на 80% навчання та 20% валідації.